In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/usr0/home/naveenr/projects/patient_provider')

In [3]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json

In [4]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import random_policy, greedy_policy
from patient.online_policies import p_approximation, p_approximation_balance, p_approximation_with_additions, p_approximation_with_additions_balance, p_approximation_with_additions_balance_learning, solve_linear_program
from patient.offline_policies import offline_solution, offline_solution_balance, offline_learning_solution
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources
from patient.learning import guess_coefficients

In [5]:
is_jupyter = 'ipykernel' in sys.modules

In [90]:
if is_jupyter: 
    seed        = 43
    num_patients = 20
    num_providers = 20
    provider_capacity = 1
    top_choice_prob = 0.5
    choice_model = "uniform_choice"
    out_folder = "policy_comparison"
    exit_option = 0.5
    num_trials = 1
    context_dim = 5
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',        help='Number of providers', type=int, default=100)
    parser.add_argument('--n_trials',          help='Number of trials ', type=int, default=2)
    parser.add_argument('--top_choice_prob',          help='Probability of picking top choice', type=float, default=0.8)
    parser.add_argument('--context_dim',          help='Context dim for patients and providers', type=int, default=5)
    parser.add_argument('--provider_capacity', help='Provider Capacity', type=int, default=5)
    parser.add_argument('--choice_model', help='Which choice model for patients', type=str, default='uniform_choice')
    parser.add_argument('--exit_option', help='What is the value of the exit option', type=float, default=0.5)
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    provider_capacity = args.provider_capacity
    top_choice_prob = args.top_choice_prob
    choice_model = args.choice_model
    exit_option = args.exit_option
    out_folder = args.out_folder
    num_trials = args.n_trials 
    context_dim = args.context_dim 

save_name = secrets.token_hex(4)  

In [91]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'top_choice_prob': top_choice_prob, 
        'choice_model': choice_model,
        'exit_option': exit_option,
        'num_trials': num_trials,
        'context_dim': context_dim} 

## Baselines

In [102]:
seed_list = [seed]
restrict_resources()

In [103]:
policy = random_policy
name = "random"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['matches'])/(num_patients*num_trials),np.mean(rewards['patient_utilities'])/(num_patients*num_trials),np.std(rewards['provider_workloads'])

Took 0.0007159709930419922 time


(0.45, 0.35719264099432857, 0.49749371855331)

In [94]:
policy = greedy_policy
name = "greedy"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['matches'])/(num_patients*num_trials),np.mean(rewards['patient_utilities'])/(num_patients*num_trials),np.std(rewards['provider_workloads'])

Took 0.0006170272827148438 time
Took 0.00044727325439453125 time
Took 0.0005402565002441406 time
Took 0.0004057884216308594 time
Took 0.0005731582641601562 time
Took 0.0007159709930419922 time
Took 0.0006372928619384766 time
Took 0.0005412101745605469 time
Took 0.0005543231964111328 time
Took 0.0005390644073486328 time
Took 0.0005414485931396484 time
Took 0.0005545616149902344 time


(0.4583333333333333, 0.38864224355835514, 0.4982608642958916)

## Online

In [95]:
policy = p_approximation
name = "p_approximation"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['matches'])/(num_patients*num_trials),np.mean(rewards['patient_utilities'])/(num_patients*num_trials),np.std(rewards['provider_workloads'])

Took 0.033118486404418945 time
Took 0.013564348220825195 time
Took 0.013570547103881836 time
Took 0.013626813888549805 time
Took 0.01407170295715332 time
Took 0.013982057571411133 time
Took 0.013829946517944336 time
Took 0.013564825057983398 time
Took 0.013807058334350586 time
Took 0.013852834701538086 time
Took 0.01365351676940918 time
Took 0.013617277145385742 time


(0.4583333333333333, 0.3832011236896941, 0.4982608642958916)

In [96]:
policy = p_approximation_balance
name = "p_approximation_balance"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['matches'])/(num_patients*num_trials),np.mean(rewards['patient_utilities'])/(num_patients*num_trials),np.std(rewards['provider_workloads'])

Took 0.036939382553100586 time
Took 0.017308473587036133 time
Took 0.01694011688232422 time
Took 0.018248319625854492 time
Took 0.016951322555541992 time
Took 0.01711440086364746 time
Took 0.017180919647216797 time
Took 0.01708531379699707 time
Took 0.01707005500793457 time
Took 0.01686835289001465 time
Took 0.01725482940673828 time
Took 0.01715874671936035 time


(0.4583333333333333, 0.3832011236896941, 0.4982608642958916)

In [97]:
policy = p_approximation_with_additions
name = "p_approximation_additions"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['matches'])/(num_patients*num_trials),np.mean(rewards['patient_utilities'])/(num_patients*num_trials),np.std(rewards['provider_workloads'])

Took 0.035620927810668945 time
Took 0.015419483184814453 time
Took 0.015078544616699219 time
Took 0.016537189483642578 time
Took 0.015310049057006836 time
Took 0.015996217727661133 time
Took 0.015866756439208984 time
Took 0.015033960342407227 time
Took 0.01538228988647461 time
Took 0.015430212020874023 time
Took 0.01540684700012207 time
Took 0.01485896110534668 time


(0.4583333333333333, 0.3856716890170183, 0.4982608642958916)

In [98]:
policy = p_approximation_with_additions_balance
name = "p_approximation_additions_balance"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['matches'])/(num_patients*num_trials),np.mean(rewards['patient_utilities'])/(num_patients*num_trials),np.std(rewards['provider_workloads'])

Took 0.03577613830566406 time
Took 0.018560409545898438 time
Took 0.01870417594909668 time
Took 0.019297361373901367 time
Took 0.018475055694580078 time
Took 0.019321918487548828 time
Took 0.019361495971679688 time
Took 0.01893472671508789 time
Took 0.018806934356689453 time
Took 0.019270658493041992 time
Took 0.018843889236450195 time
Took 0.018593311309814453 time


(0.4583333333333333, 0.3856716890170183, 0.4982608642958916)

In [99]:
policy = p_approximation_with_additions_balance_learning
name = "p_approximation_additions_balance_learning"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['matches'])/(num_patients*num_trials),np.mean(rewards['patient_utilities'])/(num_patients*num_trials),np.std(rewards['provider_workloads'])

Took 0.03954935073852539 time
Took 0.02138805389404297 time
Took 0.02159714698791504 time
Took 0.020982742309570312 time
Took 0.0206601619720459 time
Took 0.021484375 time
Took 0.021408557891845703 time
Took 0.021071195602416992 time
Took 0.020660400390625 time
Took 0.021979093551635742 time
Took 0.020539045333862305 time
Took 0.020846128463745117 time


(0.4583333333333333, 0.3840566057323451, 0.4982608642958916)

## Offline

In [100]:
policy = offline_solution
name = "offline_solution"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['matches'])/(num_patients*num_trials),np.mean(rewards['patient_utilities'])/(num_patients*num_trials),np.std(rewards['provider_workloads'])

Took 0.033345937728881836 time
Took 0.013553619384765625 time
Took 0.01352071762084961 time
Took 0.014617443084716797 time
Took 0.013826370239257812 time
Took 0.01385045051574707 time
Took 0.013760566711425781 time
Took 0.01364755630493164 time
Took 0.013838529586791992 time
Took 0.01374959945678711 time
Took 0.013604164123535156 time
Took 0.01357579231262207 time


(0.4583333333333333, 0.386520047757149, 0.4982608642958916)

In [101]:
policy = offline_learning_solution
name = "offline_learning_solution"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['matches'])/(num_patients*num_trials),np.mean(rewards['patient_utilities'])/(num_patients*num_trials),np.std(rewards['provider_workloads'])

Took 0.033899545669555664 time
Took 0.01584768295288086 time
Took 0.01600050926208496 time
Took 0.016603469848632812 time
Took 0.016025066375732422 time
Took 0.016068220138549805 time
Took 0.01612710952758789 time
Took 0.01610875129699707 time
Took 0.01608586311340332 time
Took 0.01612401008605957 time
Took 0.015980958938598633 time
Took 0.016010522842407227 time


(0.4583333333333333, 0.386191144921408, 0.4982608642958916)

In [114]:
policy = offline_solution_balance
name = "offline_solution_balance"

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.mean(rewards['matches'])/(num_patients*num_trials),np.mean(rewards['patient_utilities'])/(num_patients*num_trials),np.std(rewards['provider_workloads'])

Took 0.021645545959472656 time


(0.7, 0.49744308228226697, 0.45825756949558405)

## Save Data

In [91]:
save_path = get_save_path(out_folder,save_name)

In [92]:
delete_duplicate_results(out_folder,"",results)

In [93]:
json.dump(results,open('../../results/'+save_path,'w'))